In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dat=pd.read_csv("../input/company-bankruptcy-prediction/data.csv")

In [ ]:
t=[]
for i in range(len(dat)):     #Implementing id column which will help us during merging
    t.append(i)

dat['id']=t
dat.head()

In [ ]:
import matplotlib.pyplot as plt

n,bins,patches=plt.hist(x=dat['Bankrupt?'],bins='auto',color='#0504aa',alpha=0.7, rwidth=0.85)
plt.grid(axis='y',alpha=0.75)
plt.xlabel("value")
plt.ylabel("frequency")
plt.title("Class imbalance check!")

In [ ]:
import math
cnt_pos=0;cnt_neg=0
for i in dat['Bankrupt?']:
    if i:
        cnt_pos+=1
    else:
        cnt_neg+=1
spw = math.sqrt(cnt_pos/cnt_neg) 

In [ ]:
print("Shape of the dataset: {0}".format(dat.shape))
dat.dtypes

In [ ]:
print("## Checking NAs ##\n")
dat.isna().sum()   #High dimensional dataset, so use for loop
for i in dat.columns:
    print("NA count for *{0}* is {1}".format(i,dat[dat[i].isnull()].shape[0]))

**EDA**

In [ ]:
print("## Checking correlation ##\n")

import seaborn as sns 
sns.set_theme(style="white")
corr=dat.corr()
mask = np.triu(np.ones_like(corr, dtype=bool))
f, ax = plt.subplots(figsize=(50, 20))
cmap = sns.diverging_palette(200, 20, as_cmap=True)
sns.heatmap(corr, mask=mask, cmap=cmap, vmax=.3, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
boxplot = dat.boxplot(grid=False, column=[' ROA(C) before interest and depreciation before interest', 
                              ' Tax rate (A)', ' Net Income to Total Assets'],rot=60)

In [ ]:
dat[['Bankrupt?','id']]

Outlier Cleaning

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.covariance import EllipticEnvelope
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

y=dat[['Bankrupt?','id']]; y=y.set_index('id')
X=dat.iloc[:,1:];X=X.set_index('id')
X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.33, random_state=1)
print(X_train.shape, y_train.shape)
ee = EllipticEnvelope(contamination=0.01)  #Clean the outliers using EllipticEnvelope
yhat = ee.fit_predict(X_train)
mask = yhat != -1
X_train, y_train = X_train.loc[mask, :], y_train.loc[mask]
print(X_train.shape, y_train.shape)

In [ ]:
new_dat=X_train.merge(y_train,how='inner',on='id')

In [ ]:
new_dat2=X_test.merge(y_test,how='inner',on='id')
new_dat2.shape

In [ ]:
new_dat3=pd.concat([new_dat,new_dat2])

In [ ]:
new_dat3

In [ ]:
n,bins,patches=plt.hist(x=new_dat3[' Net Income Flag'],bins=[0.5,0.75,0.90,1,1.25,1.5],color='#0504aa',alpha=0.7, rwidth=0.85)
plt.grid(axis='y',alpha=0.75)
plt.xlabel("value")
plt.ylabel("frequency")
plt.title("What is Net Income Flag")

In [ ]:
new_dat3[' Net Income Flag'].unique()  #So we can drop it!

In [ ]:
new_dat3.drop([' Net Income Flag'],axis=1,inplace=True)

In [ ]:
for i in range(new_dat3.shape[1]):
    x_mean=new_dat3.iloc[:,i].mean()
    y_std=new_dat3.iloc[:,i].std()
    print("For column {0}-> Mean={1}, Std={2}".format(i,x_mean,y_std))


In [ ]:
n,bins,patches=plt.hist(x=(new_dat3.iloc[:,30]),color='#0504aa',alpha=0.7, rwidth=0.85)
plt.grid(axis='y',alpha=0.75)

In [ ]:
count=0
for i in new_dat3.std()<0.01:
    if i:
        count+=1
print(count)   #Columns which have less than 0.01 stdeviation (won't actually help model to predict!)

count2=0
for i in new_dat3.std()<0.015:
    if i:
        count2+=1
print(count2) #Columns which have less than 0.015 stdeviation (difference seems interesting)


In [ ]:
new_dat4=new_dat3.loc[:,new_dat3.std()>0.01]
new_dat4.shape

In [ ]:
for i in range(new_dat4.shape[1]):
    x_mean=new_dat4.iloc[:,i].mean()
    y_std=new_dat4.iloc[:,i].std()
    print("For column {0}-> Mean={1}, Std={2}".format(i,x_mean,y_std))


In [ ]:
sc_set=[10,11,13,20,28,29,32,44,45,46,47,49,51,56,57,61,65,69,70,71,72,83]  #To scale down these columns

for i in sc_set:
    dum_t=new_dat4.iloc[:,i]
    dum=[];
    for k in dum_t:
        dum.append((k-dum_t.min())/(dum_t.max()-dum_t.min()))   #You can use other than min-max scaling
                                                                #But be careful while using log transformation
    
    new_dat4.iloc[:,i]=dum

    

In [ ]:
for i in range(new_dat4.shape[1]):
    x_mean=new_dat4.iloc[:,i].mean()
    y_std=new_dat4.iloc[:,i].std()
    print("For column {0}-> Mean={1}, Std={2}".format(i,x_mean,y_std))


In [ ]:
new_dat4

*Min-max scaling may have misinterpreted column 83(Total assets to GNP price) (maybe there were outliers), check!*

In [ ]:
new_dat4=new_dat4.sort_values(by='id')

In [ ]:
plt.hist(new_dat4[' Total assets to GNP price'], bins=5)

In [ ]:
new_dat4=new_dat4.reset_index()

In [ ]:
new_dat4.drop(['id'],axis=1,inplace=True)

In [ ]:
import lightgbm
import numpy as np
import pandas as pd
from scipy import optimize
from scipy import special
from sklearn import metrics
from sklearn import model_selection

class FocalLoss:

    def __init__(self, gamma, alpha=None):
        self.alpha = alpha
        self.gamma = gamma

    def at(self, y):
        if self.alpha is None:
            return np.ones_like(y)
        return np.where(y, self.alpha, 1 - self.alpha)

    def pt(self, y, p):
        p = np.clip(p, 1e-15, 1 - 1e-15)
        return np.where(y, p, 1 - p)

    def __call__(self, y_true, y_pred):
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        return -at * (1 - pt) ** self.gamma * np.log(pt)

    def grad(self, y_true, y_pred):
        y = 2 * y_true - 1  # {0, 1} -> {-1, 1}
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        g = self.gamma
        return at * y * (1 - pt) ** g * (g * pt * np.log(pt) + pt - 1)

    def hess(self, y_true, y_pred):
        y = 2 * y_true - 1  # {0, 1} -> {-1, 1}
        at = self.at(y_true)
        pt = self.pt(y_true, y_pred)
        g = self.gamma

        u = at * y * (1 - pt) ** g
        du = -at * y * g * (1 - pt) ** (g - 1)
        v = g * pt * np.log(pt) + pt - 1
        dv = g * np.log(pt) + g + 1

        return (du * v + u * dv) * y * (pt * (1 - pt))

    def init_score(self, y_true):
        res = optimize.minimize_scalar(
            lambda p: self(y_true, p).sum(),
            bounds=(0, 1),
            method='bounded'
        )
        p = res.x
        log_odds = np.log(p / (1 - p))
        return log_odds

    def lgb_obj(self, preds, train_data):
        y = train_data.get_label()
        p = special.expit(preds)
        return self.grad(y, p), self.hess(y, p)

    def lgb_eval(self, preds, train_data):
        y = train_data.get_label()
        p = special.expit(preds)
        is_higher_better = False
        return 'focal_loss', self(y, p).mean(), is_higher_better

In [ ]:
from sklearn import model_selection
y=new_dat4[['Bankrupt?']]
y=y['Bankrupt?']
X=new_dat4.iloc[:,0:91]
fl = FocalLoss(alpha=None, gamma=0)
X_train, X_test, y_train, y_test=model_selection.train_test_split(X, y, random_state=1)

X_fit, X_val, y_fit, y_val = model_selection.train_test_split(
    X_train, y_train,
    random_state=42
)

fit = lightgbm.Dataset(
    X_fit, y_fit,
    init_score=np.full_like(y_fit, fl.init_score(y_fit), dtype=float)
)
    
val = lightgbm.Dataset(
    X_val, y_val,
    init_score=np.full_like(y_val, fl.init_score(y_fit), dtype=float),
    reference=fit
)

model = lightgbm.train(
    params={'learning_rate': 0.01},
    train_set=fit,
    num_boost_round=10000,
    valid_sets=(fit, val),
    valid_names=('fit', 'val'),
    early_stopping_rounds=20,
    verbose_eval=100,
    fobj=fl.lgb_obj,
    feval=fl.lgb_eval
)

y_pred = special.expit(fl.init_score(y_fit) + model.predict(X_test))

print()
print(f"Test's ROC AUC: {metrics.roc_auc_score(y_test, y_pred):.5f}")
print(f"Test's logloss: {metrics.log_loss(y_test, y_pred):.5f}")

In [ ]:
from tpot import TPOTClassifier
from tpot.config import classifier_config_dict
from sklearn.pipeline import make_pipeline
from xgboost import XGBClassifier
from sklearn.metrics import roc_curve,auc
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
for csbt in [0.9,1]:
    for mcw in [1,2]:
        for lr in [0.4]:
            for n in [120,200,300]:
                for spw in [8,10,12]:
                    xgb = XGBClassifier(scale_pos_weight = spw, n_estimators= n, colsample_bytree=csbt
                                        ,eval_metric = "logloss",use_label_encoder=False)
                    #xgb=xgb.fit(X_train,y_train)
                    kf = KFold(shuffle=True, n_splits=5)
                    print("mcwght: ",mcw ," l_r: ", lr  , " n: ", n, " spw: ", 
                          spw," csbt: ", csbt , " auc: ",
                          cross_val_score(xgb, X_test, y_test, cv=kf, scoring="roc_auc").mean())


**/XGB Confusion matrix with the optimized parameters\**
*scale_pos_weight=1,n_estimators=300,colsample_bytree=0.9,learning_rate=0.1,min_child_weight = 1*

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix 
xgb=XGBClassifier(scale_pos_weight=1,n_estimators=300,colsample_bytree=0.9,use_label_encoder=False,
                  eval_metric = "logloss",learning_rate=0.1,min_child_weight = 1)
xgb=xgb.fit(X_train,y_train)

plot_confusion_matrix(xgb,X_test,y_test)